In [ ]:
"""
Relevance Detection
"""

In [17]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import utils
import importlib
from collections import Counter
from sklearn.metrics import confusion_matrix
import score
from sklearn.preprocessing import StandardScaler

In [18]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)
importlib.reload(utils)

<module 'utils' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/utils.py'>

In [19]:
preprocess = preprocessing.Preprocessing()

In [20]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [21]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [22]:
stances_tr, stances_val = preprocess.train_test_split(train_bodies, train_stances)
stances_tr.shape, stances_val.shape

((39907, 3), (10065, 3))

In [23]:
# this one takes a while!
idf = preprocess.build_idf(train_bodies, stances_tr)

In [24]:
#this is just a comparison between using IDF score and not using IDF score - not related to the model
#change the body id to see
body = preprocess.get_body(5, train_bodies)
#no IDF
processed2 = preprocess.process_body(body)
print(processed2['common_nouns'],processed2['common_verbs'])

#with IDF
processed = preprocess.process_body(body, idf)
print(processed['common_nouns'],processed['common_verbs'])

AttributeError: module 'helpers' has no attribute 'clean'

In [ ]:
body = preprocess.get_body(1369, train_bodies)
processed = preprocess.process_body(body, idf)
print(processed['first_sentence']['tokens'])
print(processed['significant_sentence']['tokens'])
print(processed['first_sentence']['adverbs'],processed['significant_sentence']['adverbs'])
print(processed['first_sentence']['adjectives'],processed['significant_sentence']['adjectives'])
print(processed['first_sentence']['verbs'],processed['significant_sentence']['verbs'])

In [9]:
#this takes a while!
body_info = preprocess.process_bodies(train_bodies, idf)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
done! processed 1683


In [10]:
feats_list = [
    'shared_nouns',
    'shared_verbs',
    'shared_bigrams',
    'shared_tokens',

    'shared_nouns_fst',
    'shared_verbs_fst',
    'shared_bigrams_fst',
    'shared_tokens_fst',

    'shared_nouns_sig',
    'shared_verbs_sig',
    'shared_bigrams_sig',
    'shared_tokens_sig',

    'svo_s_fst',
    'svo_v_fst',
    'svo_o_fst',
    
    'svo_s_sig',
    'svo_v_sig',
    'svo_o_sig',
    
    'cos_nouns_sig',
    'cos_bigrams_sig',
    'cos_tokens_sig',

    'cos_nouns_fst',
    'cos_bigrams_fst',
    'cos_tokens_fst',
    
#     'sentiment_pos',
#     'sentiment_neg',
#     'sentiment_neu',
#     'sentiment_compound',
#     'sentiment_pos_fst',
#     'sentiment_neg_fst',
#     'sentiment_neu_fst',
#     'sentiment_compound_fst',
#     'sentiment_pos_sig',
#     'sentiment_neg_sig',
#     'sentiment_neu_sig',
#     'sentiment_compound_sig',
]

In [11]:
import time
# this one takes a while also! ~10 mins
start = time.time()
#train data
data_feats = [preprocess.get_feats(i, body_info) for i in stances_tr.values]
val_feats = [preprocess.get_feats(i, body_info) for i in stances_val.values]
end = time.time()
print(int(end-start))

693


In [12]:
#training data
train_df = pd.DataFrame()
for i in feats_list:
    print(i)
    train_df[i] = [x[i] for x in data_feats]

#val data
val_df = pd.DataFrame()
for i in feats_list:
    val_df[i] = [x[i] for x in val_feats]

shared_nouns
shared_verbs
shared_bigrams
shared_tokens
shared_nouns_fst
shared_verbs_fst
shared_bigrams_fst
shared_tokens_fst
shared_nouns_sig
shared_verbs_sig
shared_bigrams_sig
shared_tokens_sig
svo_s_fst
svo_v_fst
svo_o_fst
svo_s_sig
svo_v_sig
svo_o_sig
cos_nouns_sig
cos_bigrams_sig
cos_tokens_sig
cos_nouns_fst
cos_bigrams_fst
cos_tokens_fst


In [13]:
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_o_fst,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst
0,2,0,2,6,1,0,0,3,1,0,...,0,0,0,1,0.223607,0.000000,0.285714,0.141421,0.000000,0.253546
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,0,1,3,0,0,0,0,1,0,...,0,0,0,0,0.333333,0.000000,0.158114,0.000000,0.000000,0.000000
3,4,0,10,12,3,0,2,8,5,0,...,0,0,0,0,0.912871,0.667424,0.783349,0.462910,0.161165,0.596285
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
train_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_tr['Stance'])]
val_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_val['Stance'])]
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst,label
0,2,0,2,6,1,0,0,3,1,0,...,0,0,1,0.223607,0.000000,0.285714,0.141421,0.000000,0.253546,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,1,0,1,3,0,0,0,0,1,0,...,0,0,0,0.333333,0.000000,0.158114,0.000000,0.000000,0.000000,1
3,4,0,10,12,3,0,2,8,5,0,...,0,0,0,0.912871,0.667424,0.783349,0.462910,0.161165,0.596285,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [15]:
scaler = StandardScaler()

for i in feats_list:
    train_df[i] = scaler.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i] = scaler.fit_transform(val_df[i].values.reshape(-1,1))
train_df.head()

/Users/dannyyang/Library/Python/3.6/lib/python/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,shared_nouns,shared_verbs,shared_bigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_tokens_fst,shared_nouns_sig,shared_verbs_sig,...,svo_s_sig,svo_v_sig,svo_o_sig,cos_nouns_sig,cos_bigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_tokens_fst,label
0,2.696686,-0.217654,1.424817,1.626751,0.497939,-0.26567,-0.351112,1.205677,1.141655,-0.162525,...,-0.2779,-0.113102,5.562295,1.325263,-0.226840,2.137807,0.380137,-0.226485,1.129069,1
1,-0.406909,-0.217654,-0.431060,-0.746254,-0.503360,-0.26567,-0.351112,-0.551636,-0.371464,-0.162525,...,-0.2779,-0.113102,-0.179782,-0.340668,-0.226840,-0.444717,-0.442722,-0.226485,-0.551046,0
2,1.144889,-0.217654,0.496879,0.440248,-0.503360,-0.26567,-0.351112,-0.551636,1.141655,-0.162525,...,-0.2779,-0.113102,-0.179782,2.142755,-0.226840,0.984448,-0.442722,-0.226485,-0.551046,1
3,5.800281,-0.217654,8.848329,3.999757,2.500538,-0.26567,2.269755,4.134533,7.194129,-0.162525,...,-0.2779,-0.113102,-0.179782,6.460466,12.292917,6.635850,2.250716,1.451584,3.400213,1
4,-0.406909,-0.217654,-0.431060,-0.746254,-0.503360,-0.26567,-0.351112,-0.551636,-0.371464,-0.162525,...,-0.2779,-0.113102,-0.179782,-0.340668,-0.226840,-0.444717,-0.442722,-0.226485,-0.551046,0


In [16]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier
#adjust params as you see fit
model = RandomForestClassifier(n_estimators = 100, min_samples_split = 10, min_samples_leaf = 5, max_depth = 6)
model2 = LogisticRegression()
# Lasso(alpha=0.01)
# train_df.iloc[:,-1].values.reshape(-1)
model.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])
model2.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
tr_acc = model.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

tr_acc = model2.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model2.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

95.99% training accuracy
96.73% validation accuracy
95.94% training accuracy
96.66% validation accuracy


In [18]:
# get coefficients - logistic
[(feats_list[i],model2.coef_[0][i]) for i in list(range(len(feats_list)))]

#coefficients - lasso
#model.coef_

[('shared_nouns', 0.5573646966216942),
 ('shared_verbs', 0.11777636964244018),
 ('shared_bigrams', 0.9245822889852917),
 ('shared_tokens', 2.5322876891175787),
 ('shared_nouns_fst', -0.338607300708329),
 ('shared_verbs_fst', -0.06324025781339816),
 ('shared_bigrams_fst', -0.61583734062442),
 ('shared_tokens_fst', 0.4237624480560309),
 ('shared_nouns_sig', -0.01844685279714155),
 ('shared_verbs_sig', -0.10583912460891004),
 ('shared_bigrams_sig', 0.17917141394491354),
 ('shared_tokens_sig', -0.7181508272636336),
 ('svo_s_fst', 0.07697981574412302),
 ('svo_v_fst', 0.00413553379491784),
 ('svo_o_fst', 0.28120711457626346),
 ('svo_s_sig', -0.014085699682601373),
 ('svo_v_sig', -0.01825590828412707),
 ('svo_o_sig', 0.09682105801904912),
 ('cos_nouns_sig', -0.14820138720132459),
 ('cos_bigrams_sig', -0.7078592567664391),
 ('cos_tokens_sig', 1.570476798354662),
 ('cos_nouns_fst', 0.0607165030273761),
 ('cos_bigrams_fst', -0.40874983017685307),
 ('cos_tokens_fst', 1.8027646876474606)]

In [19]:
# #usage example for json dump
utils.rf_json_dump(model, list(train_df.iloc[:,:-1]), "test_rf_dump.json")

In [20]:
#dump validation data to CSV
val_df.to_csv('test_val_dump.csv')

In [21]:
true_label = val_df.iloc[:,-1]
prediction = model.predict(val_df.iloc[:,:-1])
matrix = confusion_matrix(true_label,prediction)
print('confusion matrix: \n{}\n'.format(matrix))
tn1, fp1, fn1, tp1 = matrix.ravel()

confusion matrix: 
[[7916  169]
 [ 178 2360]]



In [22]:
#use FNC scorer to generate score report
label_prediction = ["discuss" if i == 1 else "unrelated" for i in prediction]
label_actual = pd.DataFrame(stances_val)['Stance']
score.report_score(label_actual, label_prediction)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    657    |    57     |
-------------------------------------------------------------
| disagree  |     0     |     0     |    190    |    13     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |   1513    |    108    |
-------------------------------------------------------------
| unrelated |     0     |     0     |    169    |   7916    |
-------------------------------------------------------------
Score: 3703.75 out of 4559.25	(81.23594889510336%)


81.23594889510336